In [1]:
import torch

In [2]:
#https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters", len(text))

length of dataset in characters 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [97]:
chars = sorted(list(set(text)))
print(len(chars))
print(chars)
vocab_size = len(chars)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [19]:
# tokenize chars
stoi = {x:i for i,x in enumerate(chars)}
itos = {i:x for i,x in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda xs: "".join([itos[x] for x in xs])

In [21]:
print(encode('hello world'))
decode(encode('hello'))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]


'hello'

In [26]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
data[:100]

torch.Size([1115394])


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [33]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]
print(len(train_data) + len(val_data))
print(data.shape)

1115394
torch.Size([1115394])


In [35]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [48]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for i in range(block_size):
    print('Context',x[:i+1] , 'predicts', y[i].item())


Context tensor([18]) predicts 47
Context tensor([18, 47]) predicts 56
Context tensor([18, 47, 56]) predicts 57
Context tensor([18, 47, 56, 57]) predicts 58
Context tensor([18, 47, 56, 57, 58]) predicts 1
Context tensor([18, 47, 56, 57, 58,  1]) predicts 15
Context tensor([18, 47, 56, 57, 58,  1, 15]) predicts 47
Context tensor([18, 47, 56, 57, 58,  1, 15, 47]) predicts 58


In [83]:
# Add batch dimension
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def getBatch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size, (batch_size,)) ## len(data) - block_size so we don't index out of range
    x = torch.stack([data[i:block_size+i] for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    return x,y    

In [90]:
xb,yb = getBatch('train')
print('inputs')
print(xb)
print(xb.shape)
print('labels')
print(yb)
print(yb.shape)

# Spelled out
for b in range(batch_size): # Batch dimension
    for t in range(block_size):
        print('Context',xb[b,:t+1],'predicts',yb[b, t])  

inputs
tensor([[52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
        [39, 49, 12,  1, 27,  1, 58, 56]])
torch.Size([4, 8])
labels
tensor([[42,  8,  0,  0, 23, 21, 26, 19],
        [53, 42, 57,  0, 23, 43, 43, 54],
        [ 1, 61, 39, 57,  1, 51, 53, 56],
        [49, 12,  1, 27,  1, 58, 56, 39]])
torch.Size([4, 8])
Context tensor([52]) predicts tensor(42)
Context tensor([52, 42]) predicts tensor(8)
Context tensor([52, 42,  8]) predicts tensor(0)
Context tensor([52, 42,  8,  0]) predicts tensor(0)
Context tensor([52, 42,  8,  0,  0]) predicts tensor(23)
Context tensor([52, 42,  8,  0,  0, 23]) predicts tensor(21)
Context tensor([52, 42,  8,  0,  0, 23, 21]) predicts tensor(26)
Context tensor([52, 42,  8,  0,  0, 23, 21, 26]) predicts tensor(19)
Context tensor([45]) predicts tensor(53)
Context tensor([45, 53]) predicts tensor(42)
Context tensor([45, 53, 42]) predicts tensor(57)
Context tensor([45, 53, 42, 57]) predi

In [346]:
xb.shape

torch.Size([32, 8])

In [345]:
print(xb) # Input to model

tensor([[ 0, 24, 47, 51, 47, 58,  1, 43],
        [39,  1, 45, 47, 39, 52, 58,  1],
        [47, 52, 49,  1,  5, 58, 61, 47],
        [45, 46, 58, 11,  0, 13, 52, 42],
        [11,  0, 13, 52,  1, 47, 44,  1],
        [ 6,  1, 61, 46, 39, 58,  1, 57],
        [ 0,  0, 19, 24, 27, 33, 15, 17],
        [52, 43,  1, 39,  1, 54, 43, 56],
        [ 1, 52, 53, 58,  1, 44, 53, 56],
        [ 1, 42, 39, 56, 43, 42,  1, 58],
        [51, 63,  1, 53, 61, 52,  1, 50],
        [ 1, 57, 39, 63,  1, 52, 53,  6],
        [ 1, 54, 56, 53, 41, 50, 39, 47],
        [54, 54, 53, 57, 43,  1, 39, 45],
        [ 2,  1, 39,  1, 41, 53, 52, 42],
        [39, 40, 40, 50, 43,  0, 15, 39],
        [47, 56,  6,  1, 21,  1, 58, 46],
        [39, 63,  1, 58, 46, 43, 47, 56],
        [53, 44,  1, 58, 46, 43,  1, 61],
        [ 1, 47, 52,  1, 51, 43, 52, 11],
        [53, 58, 43,  2,  1, 39, 50, 50],
        [42,  6,  1, 39, 52, 42,  1, 46],
        [ 1, 63, 53, 59,  1, 51, 59, 57],
        [ 1, 25, 13, 30, 19, 13, 3

In [92]:
# Start with simplest model (bigram model) Use previous char to predict next char (counts lookup table)

In [247]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__();
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # B, T, C (C is vocab_size)
        if targets is None:
            loss = None
        else: 
            # Need to reshape for cross_entropy
            B,T,C = logits.shape
            logits = logits.view(B*T,C) # 32 65
            targets = targets.view(B*T) # 32
            loss = F.cross_entropy(logits, targets)
        return logits, loss        

    def generate(self, idx, max_tokens):
        # idx is (B, T) array of current context
        for _ in range(max_tokens):
            # get the predictions
            logits, loss= self(idx)
            # Focus on the last time dimension
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)
print('initial loss should be', -torch.log(torch.tensor(1/vocab_size)).item()) # negative log liklihood

torch.Size([256, 65])
tensor(4.8430, grad_fn=<NllLossBackward0>)
initial loss should be 4.174387454986572


In [248]:
out = model.generate(idx=torch.zeros((1,1), dtype=torch.long), max_tokens=100)
print(decode(out[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [250]:
# Create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [252]:
batch_size = 32
for steps in range(10000):
    # sample data
    xb, yb = getBatch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.4951412677764893


In [253]:
out = model.generate(idx=torch.zeros((1,1), dtype=torch.long), max_tokens=400)
print(decode(out[0].tolist()))



wr:

Man fe l in? bo houlde chaye--blllofthees s:
Aw; myoug
Thoo sduinowiu, bre hererotwestilfin ang: the s chebro lrid,
Se'lan foowom ndiske
tukistlinas:

Orat lyowe wiclin:
ALLars t th cinod as!
Thevedevead;AEJUCaroftt,
ADerde hy s tr;
GSt cicoutrncolond p f IAwathaner:
ONat they t fo be gs blle weamyoure itintet in'sut loomalll ink!
F s sur,
Thaveine thayouditelfrenevecyowe tes, hile
Fas d,
TI


In [291]:
# Self attention
B, T, C = 4, 8, 2
x = torch.randn((B, T, C))

In [292]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev,0)
    

In [293]:
print(x[0])
print(xbow[0])

tensor([[ 1.7680,  0.6972],
        [-2.0178,  0.5771],
        [ 0.9269,  0.7674],
        [-1.4343,  1.3472],
        [-1.3514, -1.2603],
        [-0.4885,  0.2505],
        [-0.4706, -0.4717],
        [ 0.3559,  0.4468]])
tensor([[ 1.7680,  0.6972],
        [-0.1249,  0.6372],
        [ 0.2257,  0.6806],
        [-0.1893,  0.8473],
        [-0.4217,  0.4257],
        [-0.4328,  0.3965],
        [-0.4382,  0.2725],
        [-0.3390,  0.2943]])


In [305]:
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b

In [306]:
print('a')
print(a)
print('b')
print(b)
print('c')
print(c)

a
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [314]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True) # Average instead of just sum (from mat mul)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b

In [315]:
print('a')
print(a)
print('b')
print(b)
print('c')
print(c)

a
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b
tensor([[9., 3.],
        [1., 9.],
        [7., 9.]])
c
tensor([[9.0000, 3.0000],
        [5.0000, 6.0000],
        [5.6667, 7.0000]])


In [334]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True) # (T, T)
xbow2 = wei @ x # x is (B ,T, C). torch applies the mat mul over the batch so it's like doing (B, T, T) @ (B, T, C) so all good => (B, T, C)

In [335]:
xbow2.shape

torch.Size([4, 8, 2])

In [336]:
torch.allclose(xbow, xbow2) # xbow2 is the same as xbow

True

In [338]:
torch.tril(torch.ones(T, T))

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [343]:
print(wei.masked_fill(tril ==0, float('-inf')))
print(tril==0)
print(F.softmax(wei.masked_fill(tril ==0, float('-inf')), dim=-1))

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[False,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False, False, False]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
    

In [337]:
# 3 version using softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril ==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [344]:
torch.arange(10)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [441]:
#version 4 self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# single head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # B, T, head_size (B, T, 16)
q = query(x) # B, T, head_size (B, T, 16)

wei = q @ k.transpose(-2,-1) # (B, T, 16) @ (B, 16, T) => (B, T, T)
wei *= head_size**-0.5 # Scale

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril ==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [459]:
torch.cat([wei, wei], -1).shape

torch.Size([4, 8, 16])

In [437]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [438]:
wei.var()

tensor(1.0918)

In [452]:
def Head(n):
    return [1]

In [464]:
[Head(1) for i in range(3)]

[[1], [1], [1]]

In [460]:
65//8

8

In [461]:
65/8

8.125

In [462]:
8*8

64

In [465]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32
block_size = 8 # context size
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32

# -----------------

torch.manual_seed(1337)

#https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# read it in to inspect it
with open('./input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
# tokenize chars
stoi = {x:i for i,x in enumerate(chars)}
itos = {i:x for i,x in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda xs: "".join([itos[x] for x in xs])


class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)  # B, T, C  (C is n_embed)
        position_emb = self.position_embedding_table(torch.arange(T, device=device)) # T, C (create an embedding for each time step)
        x = token_emb + position_emb
        logits = self.lm_head(x) # B, T, vocab_size

        if targets is None:
            loss = None
        else: 
            # Need to reshape for cross_entropy
            B,T,C = logits.shape
            logits = logits.view(B*T,C) # 32 65
            targets = targets.view(B*T) # 32
            loss = F.cross_entropy(logits, targets)
        return logits, loss        

    def generate(self, idx, max_tokens):
        
        # idx is (B, T) array of current context
        for _ in range(max_tokens):
            # get the predictions
            logits, loss= self(idx[:,-block_size:])
            # Focus on the last time dimension
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


# Data splits
data = torch.tensor(encode(text), dtype=torch.long)
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

def getBatch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size, (batch_size,)) ## len(data) - block_size so we don't index out of range
    x = torch.stack([data[i:block_size+i] for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y 

@torch.no_grad
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = getBatch(split)
            _, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean() # Average the losses to make loss less noisy
    model.train()
    return out

def printSampleFromModel(context, max_tokens):
   print(decode(model.generate(idx=context, max_tokens=max_tokens)[0].tolist()))

m = BigramLanguageModel()
model = m.to(device)

"""
LOG:
loss about 2.5
"""

'\nLOG:\nloss about 2.5\n'

In [478]:
torch.save(model, 'model.pth')

In [480]:
m = None

In [481]:
m = torch.load('model.pth')

In [482]:
m

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (position_embedding_table): Embedding(8, 32)
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)

In [484]:
m.state_dict()

OrderedDict([('token_embedding_table.weight',
              tensor([[ 0.1808, -0.0700, -0.3596,  ..., -0.8016,  1.5236,  2.5086],
                      [-0.6631, -0.2513,  1.0101,  ...,  1.5333,  1.6097, -0.4032],
                      [-0.8345,  0.5978, -0.0514,  ..., -0.4370, -1.0012, -0.4094],
                      ...,
                      [-0.6840, -0.4390, -0.7137,  ...,  1.4416, -1.8638, -1.0699],
                      [-0.6119, -0.4034,  0.3025,  ...,  0.0910,  0.5422, -0.6110],
                      [ 0.5220,  2.1368, -1.4166,  ...,  1.0078,  0.2930,  0.0943]])),
             ('position_embedding_table.weight',
              tensor([[ 0.7029,  1.4840,  0.1137, -0.7371, -1.7494,  1.6451, -1.1817, -0.6371,
                       -0.9732, -1.9146,  2.0668,  2.1432,  0.0265,  0.8326, -0.4853, -1.2051,
                       -0.7008,  2.0997,  0.1142, -0.0647,  0.5901,  0.6097, -0.7080, -0.3850,
                       -0.7042,  0.0616,  0.0651,  0.7978,  0.6647,  1.0145,  0.4815, 

In [485]:
printSampleFromModel

<function __main__.printSampleFromModel(context, max_tokens)>

In [486]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
printSampleFromModel(context, 200)


&Ks.O&HmpJQ!,!g'lN3VzFMPy'hXCchX.w.XBPlDNE.lIm!Dlnw?HxC?r?wHHVxG&DltmuyURERJFRsXUyg-NwSimQalUEavePDIYUGv&JpHBzdRO3yhCVjwy.R!yMTs-Ng3b3tB saBboP g;BxO.PNJNEMwXlDNYJpxsSFHYu3KXB.lehnsFhgRGFXhwIrlSX!?BeM


In [491]:
# ---------------- MODELS

class Head(nn.Module):
    """ Single head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # B, T, head_size (B, T, 16)
        q = self.query(x) # B, T, head_size (B, T, 16)

        wei = q @ k.transpose(-2,-1) # (B, T, 16) @ (B, 16, T) => (B, T, T)
        wei *= C**-0.5 # Scale attention
        wei = wei.masked_fill(self.tril[:T,:T] ==0, float('-inf')) # Doesn't communitcate with the past
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        out = wei @ v
        return out
    
class MultiHeadAttention(nn.Module):

    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads =nn.ModuleList([Head(head_size) for i in range(n_head)])
        self.proj = nn.Linear(n_embed, n_embed)

    def forward(self, x):
        outs = [head(x) for head in self.heads]
        out = torch.cat(outs, -1)
        out = self.proj(out)
        return out

class FeedForward(nn.Module):

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
        )
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ln1 = nn.LayerNorm(n_embed) # Normalize the last dim (C) which is n_embed
        self.ffwd = FeedForward(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = self.sa(self.ln1(x))+ x
        x = self.ffwd(self.ln2(x)) + x
        return x
    
class TransformerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(
            Block(n_embed, n_head),
            Block(n_embed, n_head),
            Block(n_embed, n_head))
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embed, vocab_size)
        

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)  # B, T, C  (C is n_embed)
        position_emb = self.position_embedding_table(torch.arange(T, device=device)) # T, C (create an embedding for each time step)
        x = token_emb + position_emb # (B, T, C)
        x = self.blocks(x)
        logits = self.lm_head(x) # B, T, vocab_size

        if targets is None:
            loss = None
        else: 
            # Need to reshape for cross_entropy
            B,T,C = logits.shape
            logits = logits.view(B*T,C) # 32 65
            targets = targets.view(B*T) # 32
            loss = F.cross_entropy(logits, targets)
        return logits, loss        

    def generate(self, idx, max_tokens):
        
        # idx is (B, T) array of current context
        for _ in range(max_tokens):
            # get the predictions
            logits, loss= self(idx[:,-block_size:])
            # Focus on the last time dimension
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [492]:
model = torch.load('model_v2.pth')
context = torch.zeros((1,1), dtype=torch.long, device=device)
printSampleFromModel(context, 400)


quid you k are rear wos to ktris?
I thano behe kinines, is bef cerege, paneefor reem me st virsereeos of dare son
Oft to more tought oryise, fotun for kind
I matper not?
Trevieelootere;
We the adife this jobsher whe,
Tcoumuss; is scromen pow jarinioned upouppaitsher; the pastanene
Be mot so,
Shrrewight.
AUTES:
Your sainrEquest are and fole peppole his cappert hom hanted is
we
conte mak admes fold

